In [2]:
from pathlib import Path

import pandas as pd

from rtnls_fundusprep.preprocessor import parallel_preprocess

## Preprocessing

This code will preprocess the images and write .png files with the square fundus image and the contrast enhanced version

This step is not strictly necessary, but it is useful if you want to run the preprocessing step separately before model inference


Create a list of files to be preprocessed:

In [3]:
ds_path = Path("../data/ODIR")
files = list((ds_path / "original").glob("*"))

Images with .dcm extension will be read as dicom and the pixel_array will be read as RGB. All other images will be read using PIL's Image.open

In [4]:
bounds = parallel_preprocess(
    files,  # List of image files
    rgb_path=ds_path / "rgb",  # Output path for RGB images
    ce_path=ds_path / "ce",  # Output path for Contrast Enhanced images
    n_jobs=4,  # number of preprocessing workers
)
df_bounds = pd.DataFrame(bounds).set_index("id")

0it [00:00, ?it/s][Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


8it [00:01,  6.07it/s][Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.9s
16it [00:02,  6.67it/s][Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.7s
20it [00:02,  7.02it/s][Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    3.5s
28it [00:04,  7.04it/s][Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.5s
36it [00:05,  7.11it/s][Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    5.8s
48it [00:07,  6.34it/s][Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.2s
56it [00:08,  6.68it/s][Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    8.8s
68it [00:09,  7.15it/s][Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   10.4s
80it [00:11,  7.04it/s][Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:   12.2s
96it [00:13,  6.93it/s][Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:   14.1s
108it [00:15,  7.06it/s][Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:   16.2s
124it [00:18,  6.76it/s][Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:

The preprocessor will produce RGB and contrast-enhanced preprocessed images cropped to a square and return a dataframe with the image bounds that can be used to reconstruct the original image. Output files will be named the same as input images, but with .png extension. Be careful with providing multiple inputs with the same filename without extension as this will result in over-written images. Any exceptions during pre-processing will not stop execution but will print error. Images that failed pre-processing for any reason will be marked with `success=False` in the df_bounds dataframe.

In [5]:
df_bounds.to_csv(ds_path / "meta.csv")